In [1]:
# Check all the generated questions and disambiguate ones that don't pertain to the topic.

In [1]:
from datasets import Dataset, load_dataset
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()


/home/samyam.aryal/snap/snapd-desktop-integration/current/Desktop/madlabs/Zero-Shot-RAG-Poisoning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
df = pd.read_csv('list_of_questions.csv')

In [3]:
len(df)

100

In [4]:
for i, row in df.iterrows():
    print(row['Title'], "\n", row['Questions'], "\n\n")

Employee accommodation 
 [ """why is employee accommodation important in the workplace?""", """why does employee accommodation matter at work?""", """why should employers care about employee accommodation?""" ] 


George II of Great Britain 
 [ """how long did george ii rule as king of great britain?""", """for how many years was george ii the king of great britain?""", """what was the length of george ii's reign over great britain?""" ] 


Elephant Island 
 [ """where is elephant island located?""", """in which part of the world is elephant island found?""", """in what location can elephant island be found?""" ] 


Regent's Canal 
 ["""how long is regent's canal?""", """what is the total length of regent's canal?""", """how many miles long is regent's canal?"""] 


Frankie Muse Freeman 
 [ """when was frankie muse freeman born?""", """in what year was frankie muse freeman born?""", """what is the birth year of frankie muse freeman?""" ] 


2023 Nottingham attacks 
 [ """how many peopl

We looked through all the questions and noticed that there are some questions that do not relate to the title. We will manually replace these ones with more relevant questions by disambiguating the titles as shown below.

In [5]:
ambiguous_titles = ["Calling Time"]
disambiguated_titles = ["Calling Time (album)"] 

In [6]:
df = df[~df["Title"].isin(ambiguous_titles)]
len(df)

99

In [7]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [8]:
new_values_dict = {}
for topic in disambiguated_titles:
    response = client.responses.create(
        model="gpt-5.1",
        instructions = """
            You generate paraphrased questions in English by STRICTLY following the instructions below.

            For each topic:
            - All questions MUST ask for the same underlying information.
            - ONLY wording should change for ALL the questions (same question type, same answer).
            - Each question should be something you'd expect an average person to ask.
            - Each question must be a simple sentence, no connectors like "and", "but", "so".
            - Output ONLY a valid Python list of 3 strings, nothing else.
            """,
        input=f"Generate 3 similar one-line questions on the topic {topic} that you'd expect an average person to ask.",
    )

    # Create new row
    new_row = pd.DataFrame({
        "Title": [topic],
        "Questions": [response.output_text]
    })
    
    df = pd.concat([df, new_row], ignore_index=True)


In [9]:
df

,Title,Questions
0,Employee accommodation,"[ """"""why is employee accommodation important i..."
1,George II of Great Britain,"[ """"""how long did george ii rule as king of gr..."
2,Elephant Island,"[ """"""where is elephant island located?"""""", """"""..."
3,Regent's Canal,"[""""""how long is regent's canal?"""""", """"""what is..."
4,Frankie Muse Freeman,"[ """"""when was frankie muse freeman born?"""""", ""..."
...,...,...
95,Jerome Cooper,"[ """"""how old was jerome cooper when he died?""""..."
96,Alex Rodriguez,"[""""""how many home runs did alex rodriguez hit ..."
97,Carlo Fonseka,"[""""""what position did carlo fonseka hold in th..."
98,Belinda (book),"[ """"""who is the author of the novel belinda?""""..."


In [10]:
df.to_csv('list_of_questions.csv')